# Основы основ

В дискретно-событийном моделировании существует всего три центральных понятия: *событие*, *процесс* и *ресурс*.
Эти понятия являются строительными блоками всего моделирования.
С их помощью можно построить модель системы любой сложности.

**Событие** — это что-либо, меняющее состояние моделируемой системы.
Например, изменение сигнала светофора приводит к смене приоритета движения на перекрёстке.

**Процесс** — это нечто продолжительное во времени, что порождает одно, несколько или бесконечное число событий.
Электромобиль заряжается на парковке.
Процесс зарядки имеет начало и заканчивается при наступлении события "аккумулятор заряжен".

**Ресурс** — это что-либо, что может использоваться разными процессами.
При этом необходимо гарантировать отсутствие [состояния гонки](https://ru.wikipedia.org/wiki/Состояние_гонки).
Ресурсом может быть, к примеру, количество колонок на автозаправочной станции, количество воды в ёмкости, товары в магазине и т. д.

При наступлении события возможно не только изменение состояния системы, но совершение *действий*, соответствующих наступившему событию.
Например, сигнал светофора сменился на разрешающий $\Rightarrow$ соответствующие автомобили поехали.

## Концепция SimPy

Информация в данном разделе взята из официальной документации.
В следующем разделе раскроем приведённые понятия подробнее.

SimPy — это библиотека дискретно-событийного моделирования.
Поведение активных компонентов (например, транспортных средств, клиентов или сообщений) моделируется с помощью процессов.
Все процессы существуют в *среде* SimPy.
Они взаимодействуют со средой и друг с другом посредством событий.

Процессы описываются простыми генераторами Python.
Вы можете называть их функцией процесса или методом процесса, в зависимости от того, является ли это обычной функцией или методом класса.
В течение своего жизненного цикла они создают события и ожидают (генерируют) их.
За ожидание отвечает оператор `yield`.

Когда процесс выдаёт (`yield`) событие, процесс приостанавливается.
SimPy возобновляет процесс после того, как событие произошло (*обработано*).
Несколько процессов могут ожидать одного и того же события.
SimPy возобновляет их в том же порядке, в котором они встали в очередь ожидания этого события.

Важным типом событий является `Timeout`.
События этого типа происходят (обрабатываются) по истечении определённого количества (модельного) времени.
Они позволяют процессу "заснуть" (удержать своё состояние) в течение заданного времени.
`Timeout`'ы и все другие события могут быть созданы путём вызова соответствующего метода `Environment`, в котором находится процесс.
Например, `Environment.timeout(...)`.

## Простая модель

Есть некоторый электромобиль, способный только парковаться и ехать.
В начальный момент времени он запаркован в течение 5 единиц времени.
Затем он едет в течение 2 единиц времени, после чего снова паркуется на 5 единиц времени.
И так до бесконечности или до заданного времени моделирования.

Вот код, реализующий дискретно-событийную модель:

In [2]:
# Импортирует SimPy
import simpy as sim

# Описываем моделируемый процесс:
# аргумент env - это экземпляр среды моделирования -
# сущность, отвечающая за всю работу с журналом событий
def car(env: sim.Environment):
    # Входим в бесконечный цикл событий
    while True:
        print(f'{env.now}: Паркуется')
        parking_duration = 5
        # Создаём событие "истечение заданного времени" (timeout)
        # и ждём (yield) его наступления
        yield env.timeout(parking_duration)
        # Событие наступило (конец парковки),
        # выполняется дальнейший код, называемый callback

        print(f'{env.now}: Едет')
        trip_duration = 2
        # Ждём событие timeout
        yield env.timeout(trip_duration)
        # Событие наступило (конец движения) -
        # цикл начинается заново

# Создаём экземпляр среды моделирования
env = sim.Environment()
# Инициализируем процесс "жизни" автомобиля
env.process(car(env))
# Запускаем основной цикл моделирования
# до заданной величины модельного времени
env.run(until=20)

0: Паркуется
5: Едет
7: Паркуется
12: Едет
14: Паркуется
19: Едет


Подробнее рассмотрим приведённый код.

*Средой* в SimPy называется сущность, отвечающая за работу с журналом событий: за инициализацию событий, определение ближайшего события, выполнение соответствующих действий (если таковые есть), за ведение счётчика модельного времени и т. д.
Тем самым мы полностью освобождены от работы с журналом событий.

В SimPy процесс описывается [*генераторной функцией*](https://pythonist.ru/generatory-python-ih-sozdanie-i-ispolzovanie/?ysclid=m1p8pbp8ww967774152) (или просто *генератором*), то есть функцией, содержащей хотя бы один оператор [`yield`](https://habr.com/ru/articles/132554/).
В остальном это такие же обычные Python-функции.

Функция `car` является генератором и на вход принимает экземпляр SimPy-среды.
Это необходимо для того, чтобы внутри этой функции иметь возможность создавать другие события и процессы.
Бесконечный цикл `while` при запуске программы не будет выполнять итерации без остановки, как в обычных функциях.
Это очевидно, если вы знакомы с генераторами: поток управления покинет функцию `car` как только встретит оператор `yield`, вернув управление среде SimPy, а затем после обработки события вновь вернувшись в функцию `car`, но уже в следующую за `yield` строку.
Подробности можете найти в веб-пособии по Python в [соответствующей главе](https://unexpectedcoder.github.io/sm6-py-cookbook/discrete-event.html).

Внутри бесконечного цикла имеется два оператора `yield`.
Для SimPy оператор `yield` синоним указания "жди наступления события".
Событие в SimPy можно создать множеством способов, каждый из которых описан в официальной документации.
Также события бывают разные.
Все их мы рассматривать здесь не будем (см. [официальную документацию](https://simpy.readthedocs.io/en/latest/topical_guides/events.html)), пока достаточно будет события [`Timeout`](https://simpy.readthedocs.io/en/latest/topical_guides/events.html#let-time-pass-by-the-timeout).
Оно создаётся с помощью метода `env.timeout(delay)`, где `delay` есть промежуток времени, через который наступит событие `Timeout`.
В данном случае у нас два варианта промежутка времени: 5 единиц для парковки и 2 единицы для движения.

При моделировании сначала ожидается событие "истечение 5 единиц времени".
После того, как оно наступает, SimPy обновляет свой счётчик времени `env.now` и выполняет callback-код, то есть код в последующих строках после `yield`.
Будет выполнена функция `print(f'{env.now}: Едет')` и создана переменная `trip_duration = 2`.
Затем создастся событие "истечение 2 единиц времени" (`env.timeout(trip_duration)`) и будет ожидаться его наступление (`yield ...`).

Затем цикл повторится.

В глобальной области видимости (в нижней части) прописано создание экземпляра среды `env = sim.Environment()`, инициализация процесса `env.process(car(env))` и запуск основного цикла моделирования `env.run(until=20)`, причём необходимо указать предел времени моделирования, иначе оно никогда бы не остановилось, ведь в процессе `car` у нас бесконечный цикл.
Обратите внимание, что в `env.process` мы передали не просто имя функции `car`, а объект генератора, созданный её вызовом `car(env)`.

```{important}
Экземпляр SimPy-среды должен быть одним единственным во всей программе, ведь несколько сред означает наличие нескольких независимых друг от друга журналов событий, а это нарушает концепцию дискретно-событийного моделирования.
```

## Заключение

Вот и всё!

На самом деле, на данном этапе не раскрыта работа с ресурсами, чтобы можно было построить модель любой системы.
Однако даже рассмотренного функционала достаточно для моделирования сложных процессов, систем и явлений.
Новые процессы и события можно создавать внутри других процессов.
Процессы могут взаимодействовать друг с другом.
События после обработки могут возвращать любые необходимые данные (значения и объекты), определённые пользователем (программистом, исследователем).

С помощью SimPy можно промоделировать любую задачу, которую вы встретите при обучении в Университете!

Однако у SimPy есть существенное ограничение для решения прикладных задач: данная библиотека не поддерживает подход параллельного программирования.
Однако существует по крайней мере пара библиотек, решающих эту проблему!
Для поиска актуальных решений воспользуйтесь поисковиком.